In [1]:
import requests
import json
import pandas as pd
from getpass import getpass
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd

C:\Users\ateso\AppData\Local\Temp\ipykernel_15544\2666485127.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [14]:
token = getpass()

········


In [35]:
def places (venue, lat, lon):
    url = f"https://api.foursquare.com/v3/places/search?query={venue}&ll={lat}%2C{lon}&limit=50"
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    response = requests.get(url, headers=headers).json()
    return response

result = places ("Stadiums", "40.4010201", "-3.7067751")

json_file_path = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Stadiums.json"

with open(json_file_path, "w") as json_file:
    json.dump(result, json_file, indent=2)

print(f"Result has been saved to {json_file_path}")


Result has been saved to C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Stadiums.json


In [ ]:
client = MongoClient("localhost:27017")
client

In [24]:
def to_csv(database_name, collection_name):
   
    client = MongoClient("localhost:27017")
    db = client[database_name]
    c = db[collection_name]

    filter_ = {
        "results.name": {"$exists": True},
        "results.geocodes.main": {"$ne": None}
    }

    projection = {
        "results.name": 1,
        "results.geocodes.main": 1
    }

    sorting = [("results.0.name", 1)]

    result_cursor = c.find(filter_, projection).sort(sorting)
    result_list = list(result_cursor)

   
    names_and_coordinates = [
        (item['name'], item['geocodes']['main']['latitude'], item['geocodes']['main']['longitude'])
        for item in result_list[0]['results']
    ]

  
    df = pd.DataFrame(names_and_coordinates, columns=['Name', 'Latitude', 'Longitude'])

  
    df.to_csv("C:\\Users\\ateso\\Desktop\\Ironhack\\Project-3\\Data\\acsv\\new_csv.csv", index=False)


In [36]:
def create_gym_map(csv_file_path):
  
    df_gym = pd.read_csv(csv_file_path)

 
    map_center = [df_gym['Latitude'].mean(), df_gym['Longitude'].mean()]

    
    gym_map = folium.Map(location=map_center, zoom_start=13)


    for index, row in df_gym.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='red', icon='dumbbell', prefix='fa')
        ).add_to(gym_map)

    return gym_map

In [39]:
def create_bars_map (csv_file_path):
    
    df_bars = pd.read_csv(csv_file_path)

    map_center = [df_bars['Latitude'].mean(), df_bars['Longitude'].mean()]

    bars_map = folium.Map(location=map_center, zoom_start=13)

    for index, row in df_bars.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='blue', icon='beer', prefix='fa')
        ).add_to(bars_map)


    return bars_map

In [40]:

def create_restaurants_map (csv_file_path):
    
    df_restaurants = pd.read_csv(csv_file_path)

    map_center = [df_restaurants['Latitude'].mean(), df_restaurants['Longitude'].mean()]

    restaurants_map = folium.Map(location=map_center, zoom_start=13)

    for index, row in df_restaurants.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='lightblue', icon='utensils', prefix='fa')
        ).add_to(restaurants_map)


    return restaurants_map

In [42]:

def create_schools_map (csv_file_path):
    
    df_schools = pd.read_csv(csv_file_path)

    map_center = [df_schools['Latitude'].mean(), df_schools['Longitude'].mean()]

    schools_map = folium.Map(location=map_center, zoom_start=16)

    for index, row in df_schools.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='lightred', icon='school', prefix='fa')
        ).add_to(schools_map)


    return schools_map

In [43]:

def create_others_map (csv_file_path):
    
    df_others = pd.read_csv(csv_file_path)

    map_center = [df_others['Latitude'].mean(), df_others['Longitude'].mean()]

    others_map = folium.Map(location=map_center, zoom_start=16)

    for index, row in df_others.iterrows():
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=row['Name'],
            icon=folium.Icon(color='orange', icon='building-columns', prefix='fa')
        ).add_to(others_map)


    return others_map

In [100]:
file_path = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Athens\Athens_all_data\Athens_data.csv"
dfAthens = pd.read_csv(file_path)

dfAthens['Local_type'] = dfAthens['Local_type'].str.capitalize()
dfAthens.sample(8)

,Name,Latitude,Longitude,Local_type
64,Pitbull Gym,37.955239,23.711949,Gym
41,Qassabah,37.976423,23.776368,Restaurants
131,Ευαγγελική Σχολή,37.931476,23.713284,Schools
288,Μέντα,38.070985,23.812476,Bar
206,K1 & K2 Oaka,38.041767,23.787371,Others
86,Yava Fitness Center - Burnazi,38.014755,23.707762,Gym
24,Naif,37.957171,23.719400,Restaurants
174,Esplanada (Εσπλανάδα),37.936128,23.691079,Others


In [1]:
mad_lat = 37.9871384
mad_lon = 23.7183808

mad_map = folium.Map(location=[mad_lat, mad_lon], zoom_start=15)


bar_group = folium.FeatureGroup(name='Bars')
gym_group = folium.FeatureGroup(name='Gyms')
school_group = folium.FeatureGroup(name='Schools')
restaurant_group = folium.FeatureGroup(name='Restaurants')
others_group = folium.FeatureGroup(name='Others')

for index, row in dfAthens.iterrows():
    Local_type = row["Local_type"]
    Name = row["Name"]
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    if Local_type == "Bar":
        icon = folium.Icon(color='blue', icon_color='black', icon='beer', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(bar_group)
    elif Local_type == "Gym":
        icon = folium.Icon(color='red', icon_color='white', icon='dumbbell', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(gym_group)
    elif Local_type == "Schools":
        icon = folium.Icon(color='lightred', icon_color='white', icon='school', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(school_group)
    elif Local_type == "Restaurants":
        icon = folium.Icon(color='lightblue', icon_color='white', icon='utensils', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(restaurant_group)
    elif Local_type == "Others":
        icon = folium.Icon(color='orange', icon_color='white', icon='building-columns', prefix='fa')
        marker = folium.Marker(
            location=[latitude, longitude],
            icon=icon,
            tooltip=f"{Local_type}: {Name}"
        )
        marker.add_to(others_group)
    else:
        print(f"Unknown Local_type: {Local_type}")
        continue


bar_group.add_to(mad_map)
gym_group.add_to(mad_map)
school_group.add_to(mad_map)
restaurant_group.add_to(mad_map)
others_group.add_to(mad_map)


folium.LayerControl().add_to(mad_map)

mad_map


NameError: name 'folium' is not defined

In [ ]:
def generate_city_map(city, lat, lon, data_path):

    df = pd.read_csv(data_path)
    df['Local_type'] = df['Local_type'].str.capitalize()

    city_map = folium.Map(location=[lat, lon], zoom_start=15)

    bar_group = folium.FeatureGroup(name='Bars')
    gym_group = folium.FeatureGroup(name='Gyms')
    school_group = folium.FeatureGroup(name='Schools')
    restaurant_group = folium.FeatureGroup(name='Restaurants')
    others_group = folium.FeatureGroup(name='Others')

    for index, row in df.iterrows():
        Local_type = row["Local_type"]
        Name = row["Name"]
        latitude = row["Latitude"]
        longitude = row["Longitude"]

        if Local_type == "Bar":
            icon = folium.Icon(color='blue', icon_color='black', icon='beer', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(bar_group)

        elif Local_type == "Gym":
            icon = folium.Icon(color='red', icon_color='white', icon='dumbbell', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(gym_group)

        elif Local_type == "Schools":
            icon = folium.Icon(color='lightred', icon_color='white', icon='school', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(school_group)

        elif Local_type == "Restaurants":
            icon = folium.Icon(color='lightblue', icon_color='white', icon='utensils', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(restaurant_group)

        elif Local_type == "Others":
            icon = folium.Icon(color='orange', icon_color='white', icon='building-columns', prefix='fa')
            marker = folium.Marker(
                location=[latitude, longitude],
                icon=icon,
                tooltip=f"{Local_type}: {Name}"
            )
            marker.add_to(others_group)

        else:
            print(f"Unknown Local_type: {Local_type}")
            continue

    bar_group.add_to(city_map)
    gym_group.add_to(city_map)
    school_group.add_to(city_map)
    restaurant_group.add_to(city_map)
    others_group.add_to(city_map)

   
    folium.LayerControl().add_to(city_map)

    return city_map

city_map = generate_city_map("Athens", 37.9871384, 23.7183808, r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Athens\Athens_all_data_300\Athens_data.csv")
city_map

In [ ]:
def create_heatmap(csv_file_path):
    df_bars = pd.read_csv(csv_file_path)

    map_center = [df_bars['Latitude'].mean(), df_bars['Longitude'].mean()]

    bars_map = folium.Map(location=map_center, zoom_start=13)

   
    locations = df_bars[['Latitude', 'Longitude']].values.tolist()

  
    HeatMap(locations).add_to(bars_map)

    return bars_map

In [113]:
df_Barce = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Game_offices\Barcelona_game_office.csv"
df_barce_off = pd.read_csv(df_Barce)

In [114]:
df_all_barce = r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Barcelona\Barcelona_all_data_300\Barcelona_data.csv"
df_barce_all = pd.read_csv(df_all_barce)

In [129]:
map_center = [df_barce_all['Latitude'].mean(), df_barce_all['Longitude'].mean()]
combined_map_barce = folium.Map(location=map_center, zoom_start=13)


marker_group = folium.FeatureGroup(name='Company to purchase').add_to(combined_map_barce)

for index, row in df_barce_off.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='dollar', prefix='fa')
    ).add_to(marker_group)


locations_all = df_barce_all[['Latitude', 'Longitude']].values.tolist()
HeatMap(locations_all, name = "Area Density").add_to(combined_map_barce)

radius = 4000
folium.Circle(location=map_center, radius=radius, color='blue', fill=True, fill_color='none', fill_opacity=0.3).add_to(combined_map_barce)

folium.LayerControl(name = "Area Density").add_to(combined_map_barce)

combined_map_barce

In [77]:
df_Paris_off = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Game_offices\Paris_games_office.csv")

In [78]:
df_Paris_all = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Paris\Paris_all_data_300\combined_paris_data.csv")

In [128]:
map_center = [df_Paris_all['Latitude'].mean(), df_Paris_all['Longitude'].mean()]
combined_map_Paris = folium.Map(location=map_center, zoom_start=13)


marker_group = folium.FeatureGroup(name='Company to purchase').add_to(combined_map_Paris)

for index, row in df_Paris_off.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='dollar', prefix='fa')
    ).add_to(marker_group)


locations_all = df_Paris_all[['Latitude', 'Longitude']].values.tolist()
HeatMap(locations_all, name = "Area Density").add_to(combined_map_Paris)

radius = 4000
folium.Circle(location=map_center, radius=radius, color='blue', fill=True, fill_color='none', fill_opacity=0.3).add_to(combined_map_Paris)

folium.LayerControl(name = "Area Density").add_to(combined_map_Paris)

combined_map_Paris

In [83]:
df_Madrid_off = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Game_offices\Madrid_games_office.csv")

In [84]:
df_Madrid_all = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Madrid\Madrid_all_data_300\combined_Madrid_data.csv")

In [126]:
map_center = [df_Madrid_all['Latitude'].mean(), df_Madrid_all['Longitude'].mean()]
combined_map_Madrid = folium.Map(location=map_center, zoom_start=13)


marker_group = folium.FeatureGroup(name='Company to purchase').add_to(combined_map_Madrid)

for index, row in df_Madrid_off.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='dollar', prefix='fa')
    ).add_to(marker_group)


locations_all = df_Madrid_all[['Latitude', 'Longitude']].values.tolist()
HeatMap(locations_all, name = "Area Density").add_to(combined_map_Madrid)

radius = 4000
folium.Circle(location=map_center, radius=radius, color='blue', fill=True, fill_color='none', fill_opacity=0.3).add_to(combined_map_Madrid)

folium.LayerControl(name = "Area Density").add_to(combined_map_Madrid)

combined_map_Madrid

In [87]:
df_Lisboa_off = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Game_offices\Lisboa_game_office.csv")

In [88]:
df_Lisboa_all = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Lisboa\Lisboa_all_300\Lisboa_data.csv")

In [125]:

map_center = [df_Lisboa_all['Latitude'].mean(), df_Lisboa_all['Longitude'].mean()]
combined_map_lisboa = folium.Map(location=map_center, zoom_start=13)


marker_group = folium.FeatureGroup(name='Company to purchase').add_to(combined_map_lisboa)

for index, row in df_Lisboa_off.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='dollar', prefix='fa')
    ).add_to(marker_group)


locations_all = df_Lisboa_all[['Latitude', 'Longitude']].values.tolist()
HeatMap(locations_all, name = "Area Density").add_to(combined_map_lisboa)

radius = 4000
folium.Circle(location=map_center, radius=radius, color='blue', fill=True, fill_color='none', fill_opacity=0.3).add_to(combined_map_lisboa)

folium.LayerControl(name = "Area Density").add_to(combined_map_lisboa)

combined_map_lisboa

In [90]:
df_Athens_off = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Game_offices\Athens_game_office.csv")

In [91]:
df_Athens_all = pd.read_csv(r"C:\Users\ateso\Desktop\Ironhack\Project-3\Data\Athens\Athens_all_data_300\Athens_data.csv")

In [124]:

map_center = [df_Athens_all['Latitude'].mean(), df_Athens_all['Longitude'].mean()]
combined_map_Athens = folium.Map(location=map_center, zoom_start=13)


marker_group = folium.FeatureGroup(name='Company to purchase').add_to(combined_map_Athens)

for index, row in df_Athens_off.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Name'],
        icon=folium.Icon(color='green', icon='dollar', prefix='fa')
    ).add_to(marker_group)


locations_all = df_Athens_all[['Latitude', 'Longitude']].values.tolist()
HeatMap(locations_all, name = "Area Density").add_to(combined_map_Athens)

radius = 4000  
folium.Circle(location=map_center, radius=radius, color='blue', fill=True, fill_color='none', fill_opacity=0.3).add_to(combined_map_Athens)

folium.LayerControl(name = "Area Density").add_to(combined_map_Athens)

combined_map_Athens